In [1]:
import pandas as pd
import os
import re
import numpy as np
from string_search import *

N_GRAM_SIZE = 4

data_dir = r'C:\projects\personal\kaggle\kaggle_coleridge_initiative\string_search\data'
data_path = os.path.join(data_dir, 'gov_data.csv')

def text_cleaning(text):
    text = re.sub('[^A-Za-z]+', ' ', str(text)).strip() # remove unnecessary literals

    # remove extra spaces
    text = re.sub("\s+"," ", text)

    return text.lower().strip()

    import string

def is_name_ok(text):
    if len([c for c in text if c.isalnum()]) < 4:
        return False
    if len(text.split(' ')) < 3:
        return False

    return True

def get_n_grams(s):
    s = f' {s} '
    n_grams = []
    for i in range(len(s) - N_GRAM_SIZE + 1):
        ngram_str = s[i: i + N_GRAM_SIZE]
        #if ' ' not in ngram_str[1:]:
        if not ' ' in ngram_str[1:3]:
            n_grams.append(ngram_str)
        
    
    return reduce_n_grams(n_grams)

def reduce_n_grams(n_grams):
    n_grams_single = []
    for token in n_grams:
        if token not in n_grams_single:
            n_grams_single.append(token)
    return n_grams_single

##### STEP 1: Make a list of the known labels provided to us

metadata = pd.read_csv(r'C:\projects\personal\kaggle\kaggle_coleridge_initiative\data\train.csv')
temp_1 = [text_cleaning(x) for x in metadata['dataset_label']]
temp_2 = [text_cleaning(x) for x in metadata['dataset_title']]
temp_3 = [text_cleaning(x) for x in metadata['cleaned_label']]

existing_labels = temp_1 + temp_2 + temp_3
existing_labels = [l.lower() for l in existing_labels]
existing_labels = list(set(existing_labels))
# Sort labels by length in descending order
existing_labels = sorted(existing_labels, key = len, reverse= True)

In [21]:
df = pd.read_csv(data_path)

df = pd.concat([df, pd.DataFrame({'title': existing_labels})], ignore_index= True).reset_index(drop = True)
#df['raw'] = df.title
df['title'] = df.title.apply(text_cleaning)

print(len(df))
to_remove = ['data set', 'research reports', 'data science', 'council c', 'line p', 'point lay', 'l surf', 'analysis data', 'test results', 'annual report', 'ap index', 'cyber security', 'rural residents', 'financial information', 'invasive species', 'strategic plan', 'administrative data', 'supporting information', 'domestic violence']
titles = list(df.title.unique())
for t in to_remove:
    titles.remove(t)

df = pd.DataFrame({'title': titles})

print(len(df))
df = df.loc[df.title.apply(is_name_ok)]

print(len(df))

#df = df.iloc[:1000]

292115
139120
137647


In [3]:
df.head()

,title
0,department for the aging dfta geriatric mental...
1,low altitude aerial imagery obtained with unma...
2,forestry planting spaces
3,nys math test results by grade citywide by rac...
4,high operational temperature mwir detectors wi...


## Get N-Grams

In [4]:
df_ngram = pd.DataFrame()
for col in df.columns:
    df_ngram[col] = df[col].apply(lambda x: get_n_grams(x))

In [5]:
df_ngram.head()

,title
0,"[ dep, depa, epar, part, artm, rtme, tmen, men..."
1,"[ low, low , alt, alti, ltit, titu, itud, tud..."
2,"[ for, fore, ores, rest, estr, stry, try , pl..."
3,"[ nys, nys , mat, math, ath , tes, test, est..."
4,"[ hig, high, igh , ope, oper, pera, erat, rat..."


## Create Index

In [6]:
import math

def get_n_gram_length(x):
    return max(1, len(x))

name_index = create_ngram_index(df_ngram['title'].values)
name_lens = df_ngram['title'].apply(get_n_gram_length).values

## Search

In [7]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import re
import seaborn as sns
from tqdm import tqdm
import nltk
import random
from nltk.tokenize import word_tokenize,sent_tokenize
import pickle

train_example_names = [fn.split('.')[0] for fn in os.listdir(r'C:\projects\personal\kaggle\kaggle_coleridge_initiative\data\train')]

docIdx = train_example_names.copy()

def load_train_example_by_name(name):
    doc_path = os.path.join(r'C:\projects\personal\kaggle\kaggle_coleridge_initiative\data\train', name + '.json')
    with open(doc_path) as f:
        data = json.load(f)
    return data

def get_doc(doc_id):
    doc_json = load_train_example_by_name(doc_id)
    doc_text = ' '.join([sec['text'] for sec in doc_json])

    # Tokenize sentencewise
    sentences = sent_tokenize(doc_text)
    sentences = [text_cleaning(s) for s in sentences]
    return sentences

In [8]:
with open(r'C:\projects\personal\kaggle\kaggle_coleridge_initiative\data/bert_ner_data/pos.pkl', 'rb') as f:
    pos_sentences_processed = pickle.load(f)

In [9]:
def get_matches(input_n_grams, n_gram_index, n_gram_lengths):
    input_n_gram_set = input_n_grams
    input_len = len(input_n_gram_set)
    matches = {}

    for token in input_n_gram_set:
        if token in n_gram_index:
            token_matches = n_gram_index[token]

            for token_match in token_matches:
                if token_match in matches:
                    # Increase matching token count of this entry
                    matches[token_match] += 1
                else:
                    # Add new entry
                    matches[token_match] = 1

    #matches = {k: v / max(input_len, n_gram_lengths[k]) for k,v in matches.items()}
    matches = {k: v / input_len for k,v in matches.items()}

    return [(k, v) for k, v in matches.items()]


def get_matches(input_n_grams, n_gram_index, n_gram_lengths):
    input_n_gram_set = input_n_grams
    input_len = len(input_n_gram_set)
    matches = {}

    for token in input_n_gram_set:
        if token in n_gram_index:
            token_matches = n_gram_index[token]

            for token_match in token_matches:
                if token_match in matches:
                    # Increase matching token count of this entry
                    matches[token_match] += 1
                else:
                    # Add new entry
                    matches[token_match] = 1

    #matches = {k: v / max(input_len, n_gram_lengths[k]) for k,v in matches.items()}
    matches = {k: v / n_gram_lengths[k] for k,v in matches.items()}
    #matches = {k: v / input_len for k,v in matches.items()}

    return [(k, v) for k, v in matches.items()]

def get_match_values(matches, entries):
    return [(entries[i], score) for i, score in matches]

def get_sentence_matches(s, name_index, name_lens, names):
    input_ngrams = get_n_grams(s)
    matches = get_matches(input_ngrams, name_index, name_lens)
    matches = [(i, score) for i, score in matches if score >= MATCH_THRESHOLD]
    matches = get_match_values(matches, names)
    matches = [(text, score) for text, score in matches if f' {text.lower()} ' in f' {s} ']
    return [text for text, _ in matches]

MATCH_THRESHOLD = 0.9
def get_doc_matches(sentences):
    doc_matches = []
    for s in tqdm(sentences):
        matches = get_sentence_matches(s, name_index, name_lens, df.title.values)
        doc_matches.extend(matches)

    return list(set(doc_matches))

In [22]:
doc_id = docIdx[26]
sentences = get_doc(doc_id)
matches = [l for l in df.title.values if l in ' '.join(sentences)]#matches = get_doc_matches(sentences)

In [23]:
matches

['high schools',
 'national education longitudinal study',
 'education longitudinal study']

In [16]:
[s for s in sentences if 'domestic violence' in s]

[]

In [24]:
to_remove = ['data set', 'research reports', 'data science', 'council c', 'line p', 'point lay', 'l surf', 'analysis data', 'test results', 'annual report', 'ap index', 'cyber security', 'rural residents', 'financial information', 'invasive species', 'strategic plan', 'administrative data', 'supporting information', 'domestic violence', 'high schools']